# 决策树学习
决策树是一种常用的监督学习算法，适用于分类和回归任务。

In [5]:
# 导入必要的库
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# 加载数据集
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# 创建并训练决策树模型
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# 测试模型
accuracy = clf.score(X_test, y_test)
print(f'模型准确率: {accuracy:.2f}')

模型准确率: 1.00


## 信息熵、信息增益率和基尼系数
在决策树中，信息熵用于衡量数据的不确定性，信息增益率用于选择最优划分属性，基尼系数用于评估数据集的纯度。

## 信息熵、信息增益率和基尼系数的定义和公式
### 信息熵 (Entropy)
信息熵是衡量数据集不确定性的一种指标，其公式为：
$$ Entropy = -\sum_{i=1}^{n} p_i \log_2(p_i) $$
其中，$p_i$ 是类别 $i$ 的概率。

### 信息增益率 (Information Gain Ratio)
信息增益率用于衡量某一特征对数据集划分的有效性，其公式为：
$$ GainRatio = \frac{Gain}{IV} $$
其中，$Gain$ 是信息增益，$IV$ 是固有值 (Intrinsic Value)。

### 基尼系数 (Gini Index)
基尼系数用于衡量数据集的纯度，其公式为：
$$ Gini = 1 - \sum_{i=1}^{n} p_i^2 $$
其中，$p_i$ 是类别 $i$ 的概率。

In [6]:
# 计算信息熵
#当数据量一定时，系统越有序，熵值越低，系统越混乱，熵值越高
from math import log2

def entropy(labels):
    total = len(labels)
    counts = {label: labels.count(label) for label in set(labels)}
    return -sum((count / total) * log2(count / total) for count in counts.values())

# 示例数据
labels = [0, 0, 1, 1, 1, 0, 1]
print(f'信息熵: {entropy(labels):.4f}')

信息熵: 0.9852


In [7]:
# 计算基尼系数
def gini(labels):
    total = len(labels)
    counts = {label: labels.count(label) for label in set(labels)}
    return 1 - sum((count / total) ** 2 for count in counts.values())

# 示例数据
print(f'基尼系数: {gini(labels):.4f}')

基尼系数: 0.4898


In [8]:
# 计算信息增益率
def information_gain_ratio(parent_entropy, subsets):
    total = sum(len(subset) for subset in subsets)
    weighted_entropy = sum((len(subset) / total) * entropy(subset) for subset in subsets)
    intrinsic_value = -sum((len(subset) / total) * log2(len(subset) / total) for subset in subsets if len(subset) > 0)
    gain = parent_entropy - weighted_entropy
    return gain / intrinsic_value if intrinsic_value != 0 else 0

# 示例数据
subset1 = [0, 0, 1]
subset2 = [1, 1, 0, 1]
parent_entropy = entropy(labels)
gain_ratio = information_gain_ratio(parent_entropy, [subset1, subset2])
print(f'信息增益率: {gain_ratio:.4f}')

信息增益率: 0.1300


## 篮球比赛熵值计算
假设有4个球队 {A, B, C, D}，它们的获胜概率分别为 {1/2, 1/4, 1/8, 1/8}，计算其熵值。

In [9]:
# 计算熵值
probabilities = [1/2, 1/4, 1/8, 1/8]
entropy_value = -sum(p * log2(p) for p in probabilities)
print(f'篮球比赛的熵值 Ent(D): {entropy_value:.4f}')

篮球比赛的熵值 Ent(D): 1.7500
